In [2]:
import pandas as pd

In [3]:
# What does the data look? 
data = pd.read_csv('datasets/output_all_wide_502.csv')
print(data.shape)
data.head(n=3)


(1317, 21602)


,PID,Start Timestamp,aEEG-1,aEEG-2,aEEG-3,aEEG-4,aEEG-5,aEEG-6,aEEG-7,aEEG-8,...,aEEG-21591,aEEG-21592,aEEG-21593,aEEG-21594,aEEG-21595,aEEG-21596,aEEG-21597,aEEG-21598,aEEG-21599,aEEG-21600
0,PUH-2010-078,2010-05-31T05:45:22Z,4.822085,4.459995,3.472875,3.465390,3.530755,3.368620,3.878010,4.335245,...,2.991655,3.224170,5.79905,6.060715,6.768355,4.697730,4.061875,4.50477,4.783415,5.165005
1,PUH-2010-014,2010-10-12T15:06:04Z,3.772575,3.112210,3.396625,4.686235,5.627865,5.483825,4.686395,3.572505,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PUH-2010-080,2010-06-02T22:38:15Z,2.218955,2.662795,2.167510,1.911485,4.196755,2.719905,1.784490,3.090455,...,6.107765,3.842555,2.14388,2.465135,5.171495,4.613265,2.652210,2.38732,4.571230,5.408405


In [4]:
# Exploring the meta field
path = 'datasets/output_all_meta_502.csv'
data_meta = pd.read_csv(path)

print('Meta data shape: ', data_meta.shape)
data_meta.head(n = 3)

Meta data shape:  (1317, 8)


,PID,Export start time,Export end time,Number of data segments,Duration in seconds,Number of data used,Occurance insufficient data,Should considered as good
0,PUH-2010-078,2010-05-31T05:45:22Z,2010-05-31T11:45:22Z,1,133721,21600,0,Yes
1,PUH-2010-014,2010-10-12T15:06:04Z,2010-10-12T21:06:04Z,1,1404,1375,0,Yes
2,PUH-2010-080,2010-06-02T22:38:15Z,2010-06-03T04:38:15Z,3,166561,21600,0,Yes


In [ ]:
# Read the expert labels
path = 'datasets/eeg PUH mater eeg file _deidentified.xlsx'
expertLabels = pd.read_excel(path, sheet_name = "Sheet1")

# Drop all rows for which all columns are NaN
expertLabels.dropna(axis = 'index', how = 'all', inplace = True)

# The same values are not repeated and therefore NaNs. Replace NaNs with the appropriate labels

# Whether the present columns represents the start or end of a study
expertLabels['Study start (Bool)'] = expertLabels['Study start/End'].notna()

# ids same as the non-NaN id above
expertLabels[['id']] = expertLabels[['id']].fillna(method = 'ffill', axis = 'index')

# timestamp is the same as End if NaN
expertLabels[['Study start/End', 'Timestamp']] = expertLabels[['Study start/End', 'Timestamp']].fillna(method = 'ffill', 
                                                                                                       axis = 'columns')
# drop the study start end column
expertLabels.drop(columns = ['Study start/End'], inplace = True)

# Rename columns with large names
expertLabels.rename(columns={'Background: Background: 1 = suppressed; 2 = suppression-burst  3 = continuous with periods of attenuation 4 - continuous': 'Background', 
                             'Superimposed patterns:  0 - Nothing (suppressed) 1 - Seizure (convulsive or nonconvulsive); 2 - Myoclonic status; 3 - polyspike-wave; 4 -GPED; 5- non-GPED periodic patern; 6 - epileptiform discharges; 7 - nothing epileptiform. 8-GPED-Seizure': 'Superimposed patterns', 
                             'Reactivity: 0- No; 1- Yes': "Reactivity"}, inplace = True)

# drop rows for which background and superimposed patterns are NaNs
expertLabels.dropna(axis = 'index', how = 'all', subset = ['Background', 'Superimposed patterns'], inplace = True)

# Now fill NaNs in the reactivity column
expertLabels[['Reactivity']] = expertLabels[['Reactivity']].fillna(method = 'ffill', axis = 'index')

# Filter expert labels: focus on: background ==2 AND superimposed = one of 2, 3, 4, 5 or 8 
# (these can be collapsed into a single category)
# expertLabels = expertLabels.loc[(expertLabels['Background'] == 2) & (expertLabels['Superimposed patterns'].isin([2, 3, 4, 5, 8]))]

# How do the expert labels look?
print('Shape of Expert labels: ', expertLabels.shape)
# expertLabels.head(n=5)